In [ ]:
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import cv2
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import train_test_split
from torchvision import transforms
from PIL import Image
import os
import unet
import dice

# Check if GPU is available
availability = torch.cuda.is_available()

# Split the data into train set and validation set 
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')  
df['ImageId'], df['ClassId'] = zip(*df['ImageId_ClassId'].str.split('_'))
df['ClassId'] = df['ClassId'].astype(int)
df = df.pivot(index='ImageId',columns='ClassId',values='EncodedPixels')
df['defects'] = df.count(axis=1)
train_data, val_data = train_test_split(df, test_size=0.3, random_state=42)
train_data.to_csv('train_1.csv', index=False)
val_data.to_csv('val_1.csv', index=False)

# Only use images with defects for training
train_data = train_data[train_data["defects"] > 0]


# Convert the encoding of a single type of defect to an array
def encoding2layer(enc):
    label = np.zeros(256 * 1600, dtype = int)
    if pd.isnull(enc):
        return label.reshape((256,1600))
    defects = [int(x) for x in enc.split(" ")]
    i = 0
    while i < len(defects):
        # 1-indexed
        label[defects[i] - 1 : defects[i] + defects[i + 1] - 1] = 1
        i += 2
    label = label.reshape((1600, 256))
    label = np.rot90(label)
    label = np.flipud(label)
    return label

# Convert the encoding of the 4 defects to an array of (4, 256, 1600)
def encoding2label(enc1, enc2, enc3, enc4):
    result = np.zeros((4, 256, 1600), dtype = int)
    result[0,:,:] = encoding2layer(enc1)
    result[1,:,:] = encoding2layer(enc2)
    result[2,:,:] = encoding2layer(enc3)
    result[3,:,:] = encoding2layer(enc4)
    return result

# Define the Dataset class
class DefectDataset(Dataset):
    def __init__(self, data, root_dir, datatype = "train"):
        self.data = data
        self.root_dir = root_dir
        self.datatype = datatype
    
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        image_name = self.data.iloc[index].name
        image = Image.open(self.root_dir + image_name)
        image = cv2.imread(self.root_dir + image_name)
        image = image.transpose((2,0,1))
        image_tensor = torch.Tensor(image)
        if self.datatype == "train":
            label = encoding2label(self.data.iloc[index, 0], self.data.iloc[index, 1], self.data.iloc[index, 2], self.data.iloc[index, 3])
            label_tensor = torch.from_numpy(label)
            return image_tensor, label_tensor
        else:
            return image_tensor

# Data loader 
train_set = DefectDataset(train_data, '../input/severstal-steel-defect-detection/train_images/')
train_loader = DataLoader(train_set, batch_size = 4,
                        shuffle = True, num_workers=4)
val_set = DefectDataset(val_data, '../input/severstal-steel-defect-detection/train_images/')
val_loader = DataLoader(val_set, batch_size = 4,
                        shuffle = False, num_workers=4)


In [ ]:
# loss function: Dice coefficient loss
def diceLoss(predict, label):
    m = nn.Softmax2d()
    #probability = F.softmax(predict, dim = 1)
    probability = m(predict)
    probability = probability[:, 1:5, :, :]
    smooth = 1
    iflat = probability.contiguous().view(-1)
    tflat = label.contiguous().view(-1)
    intersection = (iflat * tflat).sum()
    dice = (2 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth)
    return 1 - dice
 

In [ ]:
model = unet.unet()

if availability:
    model = model.cuda()

loss_func = diceLoss
optimizer = optim.Adam(model.parameters(), lr = 5e-4)
BATCH_SIZE = 4
max_iter = 20 # maximum number of iterations
current_loss = float("inf")
current_dice = 0

# loss of training and evaluation set and dice coefficient of evaluation set
train_loss = []
val_loss = []
positive = []
negative = []

# Train the model
for epoch in range(max_iter):
    
    epoch_loss = 0
    epoch_loss_val = 0
    
    for step, (image, label) in enumerate(train_loader):
        if availability:
                image = image.cuda()
                label = label.cuda()
        model.train()
        prediction = model(image)
        loss = loss_func(prediction, label)
        epoch_loss += loss.item() * prediction.shape[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    epoch_loss /= len(train_set)
    train_loss.append(epoch_loss)
    
    # test the result on the validation set
    epoch_pos = []
    epoch_neg = []
    with torch.no_grad():
        for step, (image, label) in enumerate(val_loader):
            if availability:
                image = image.cuda()
                label = label.cuda()
            model.eval()
            prediction = model(image)
            loss = loss_func(prediction, label)
            epoch_loss_val += loss.item() * prediction.shape[0]
            
            if availability:
                prediction = prediction.cpu().detach().numpy()
            else:
                prediction = prediction.detach().numpy()
            if availability:
                label = label.cpu().detach().numpy()
            else:
                label = label.detach().numpy()
            for i in range(prediction.shape[0]):
                pos, neg = dice.dice(prediction[i, :, :, :], label[i, :, :])
                epoch_pos.append(pos)
                epoch_neg.append(neg)
            
           
        epoch_loss_val /= len(val_set)
        val_loss.append(epoch_loss_val)
        dice_positive = sum(epoch_pos) / len(epoch_pos)
        dice_negative = sum(epoch_neg) / len(epoch_neg)
        positive.append(dice_positive)
        negative.append(dice_negative)
        
        # Save the model if the dice coefficient is higher
        if (dice_positive > current_dice):
            current_dice = dice_positive
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': current_loss
            }, "./model_dice.pth")

    # Print the loss and dice coefficient of each epoch
    print("Current epoch", epoch, "  train loss", epoch_loss, "  val loss", epoch_loss_val, "  Dice positive", dice_positive, "  Dice negative", dice_negative)

# Save the data
train_loss = np.array(train_loss)
np.savetxt("train_loss_dice.txt", train_loss)
val_loss = np.array(val_loss)
np.savetxt("val_loss_dice.txt", val_loss)
positive = np.array(positive)
negative = np.array(negative)
np.savetxt("positive_dice.txt", positive)
np.savetxt("negative_dice.txt", negative)